# Strip PDFs and export text to csv file
- I wrote this function to automate the process of copying and pasting pdf files for a business

In [1]:
working_directory = ('whatever your working directory needs to be ')
def pdf_text_to_csv(rootdir):
    
    #TODO: update with method for extracting specific page numbers
    
    '''iterates through a folder directory, extracts text from PDFs, converts the distinct text to a dataframe,
    and exports the dataframe as a csv file with name - 'text_df' - 
     arguments: rootdir = filepath to the folder you want the function to iterate through'''
    import fitz
    import os
    
    os.chdir(working_directory)
    
    text_list = []
    
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            doc = fitz.open(file)
            page = doc[0]
            text = page.getText("text")                    
                
            text_list.append(text)            
            doc.close()
            df = pd.DataFrame(text_list, columns = ['text'])
            df.to_csv('text_df.csv')                         #change name of csv file if you wish
    return df

# EDA Functions

In [141]:
#this function brings columns to wherever you wnat, 
# can be useful https://stackoverflow.com/questions/13148429/how-to-change-the-order-of-dataframe-columns
def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]

In [ ]:
#displays tables side-by-side https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side
from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

In [ ]:
# Printing the percentage of missing values per column
def percent_missing(dataframe):
    '''
    Prints the percentage of missing values for each column in a dataframe
    '''
    # Summing the number of missing values per column and then dividing by the total
    sumMissing = dataframe.isnull().values.sum(axis=0)
    pctMissing = sumMissing / dataframe.shape[0]
    
    if sumMissing.sum() == 0:
        print('No missing values')
    else:
        # Looping through and printing out each columns missing value percentage
        print('Percent Missing Values:', '\n')
        for idx, col in enumerate(dataframe.columns):
            if sumMissing[idx] > 0:
                print('{0}: {1:.2f}%'.format(col, pctMissing[idx] * 100))

In [ ]:
#https://stackoverflow.com/questions/29294983/how-to-calculate-correlation-between-all-columns-and-remove-highly-correlated-on
def filter_df_corr(inp_data, corr_val):
    '''
    Returns an array or dataframe (based on type(inp_data) adjusted to drop \
        columns with high correlation to one another. Takes second arg corr_val
        that defines the cutoff

    ----------
    inp_data : np.array, pd.DataFrame
        Values to consider
    corr_val : float
        Value [0, 1] on which to base the correlation cutoff
    '''
    # Creates Correlation Matrix
    if isinstance(inp_data, np.ndarray):
        inp_data = pd.DataFrame(data=inp_data)
        array_flag = True
    else:
        array_flag = False
    corr_matrix = inp_data.corr()

    # Iterates through Correlation Matrix Table to find correlated columns
    drop_cols = []
    n_cols = len(corr_matrix.columns)

    for i in range(n_cols):
        for k in range(i+1, n_cols):
            val = corr_matrix.iloc[k, i]
            col = corr_matrix.columns[i]
            row = corr_matrix.index[k]
            if abs(val) >= corr_val:
                # Prints the correlated feature set and the corr val
                print(col, "|", row, "|", round(val, 2))
                drop_cols.append(col)
                
    #print(f'Highly Correlated Columns: {drop_cols}')
    # Drops the correlated columns (you can also just have this function print the highly correlated columns)
    drop_cols = set(drop_cols)
    inp_data = inp_data.drop(columns=drop_cols)
    # Return same type as inp
    if array_flag:
        return inp_data.values
    else:
        return inp_data

In [11]:
#mine
def z_score_outliers(x, threshold):
    
    '''returns index of outliers and their values as zip object
    arguments: column of dataframe as x, threshold of standard deviations (typically 3) as z-score threshold
    '''
    import numpy as np
    from scipy import stats
    z = np.abs(stats.zscore(x))
    outliers = x[z > threshold]
    outliers_index = x[z > threshold].index
    outlier_pairs = zip(outliers_index, outliers)
    return [x for x in outlier_pairs]

In [ ]:
#Another function I found, but I think I like mine more (the last one), threshold isn't active in this function
def z_score_indices_of_outliers(X, threshold=3):
    '''
    Detects outliers using Z-score standardization
    
    Input: - X: A feature in your dataset
           - threshold: The number of standard deviations from the mean
                        to be considered an outlier
                        
    Output: A data frame with all outliers beyond 3 standard deviations
    '''
    X_mean = np.mean(X)
    X_stdev = np.std(X)
    z_scores = [(y - X_mean) / X_stdev for y in X]
    z_df = pd.DataFrame(z_scores)
    pos_outliers = z_df[z_df[0] > 3]
    neg_outliers = z_df[z_df[0] < -3]
    return pos_outliers; neg_outliers

In [ ]:
#flatten nested jsons
# https://towardsdatascience.com/how-to-flatten-deeply-nested-json-objects-in-non-recursive-elegant-python-55f96533103d
def flatten_json(nested_json):
    """
        Flatten json object with nested keys into a single level.
        Args:
            nested_json: A nested json object.
        Returns:
            The flattened json object if successful, None otherwise.
    """
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(nested_json)
    return out

In [ ]:
#flatten DEEPLY nested JSON, same source as above
from itertools import chain, starmap

def flatten_json_iterative_solution(dictionary):
    """Flatten a nested json file"""

    def unpack(parent_key, parent_value):
        """Unpack one level of nesting in json file"""
        # Unpack one level only!!!
        
        if isinstance(parent_value, dict):
            for key, value in parent_value.items():
                temp1 = parent_key + '_' + key
                yield temp1, value
        elif isinstance(parent_value, list):
            i = 0 
            for value in parent_value:
                temp2 = parent_key + '_'+str(i) 
                i += 1
                yield temp2, value
        else:
            yield parent_key, parent_value    

            
    # Keep iterating until the termination condition is satisfied
    while True:
        # Keep unpacking the json file until all values are atomic elements (not dictionary or list)
        dictionary = dict(chain.from_iterable(starmap(unpack, dictionary.items())))
        # Terminate condition: not any value in the json file is dictionary or list
        if not any(isinstance(value, dict) for value in dictionary.values()) and \
           not any(isinstance(value, list) for value in dictionary.values()):
            break

    return dictionary

## Time Series Functions

In [ ]:
from statsmodels.tsa.stattools import adfuller

def adf_test(series,title=''):
    """
    Pass in a time series and an optional title, returns an ADF report
    """
    print(f'Augmented Dickey-Fuller Test: {title}')
    result = adfuller(series.dropna(),autolag='AIC') # .dropna() handles differenced data
    
    labels = ['ADF test statistic','p-value','# lags used','# observations']
    out = pd.Series(result[0:4],index=labels)

    for key,val in result[4].items():
        out[f'critical value ({key})']=val
        
    print(out.to_string())          # .to_string() removes the line "dtype: float64"
    
    if result[1] <= 0.05:
        print("Strong evidence against the null hypothesis")
        print("Reject the null hypothesis")
        print("Data has no unit root and is stationary")
    else:
        print("Weak evidence against the null hypothesis")
        print("Fail to reject the null hypothesis")
        print("Data has a unit root and is non-stationary")

# Survey Monkey Specific API Functions

In [ ]:
#I created this to grab the headings from JSON survey data pulled from the SurveyMonkey API, email me if you ever do this, its a journey!

json = client.get_survey_details('insert survey id')

def get_headings(json):
    
    '''
    
    This function takes a SurveyMonkey json object from an API call and creates a dataframe with the headers and the corresponding question_ids
    
    Arguments: a single json object created from a .get() call on the SurveyMonkey API
    
    requirements: SurveyMonkey client - https://github.com/GearPlug/surveymonkey-python
    '''
    
    
    ids_list = []
    for pages in json['pages']:
        for question in pages['questions']:
            for ids in question['id']:
                ids_list.append(ids)     
    def divide_chunks(l, n): 
      
    # looping till length l 
        for i in range(0, len(l), n):  
            yield l[i:i + n] 
  
    # How many elements each 
    # list should have 
    n = 9
  
    x = list(divide_chunks(ids_list, n)) 

    #create question id dataframe from list of ids the join is joining together each list into one number, taking out commas and quotation marks
    heading_ids = pd.DataFrame([''.join(i) for i in x])

    headings_list = []
    for pages in json['pages']:
        for question in pages['questions']:
            for headings in question['headings']:
                headings_list.append(headings)
                headings = pd.DataFrame(headings_list)

#concatenate the heading ids with the headings
    questions_and_ids = pd.concat([headings, heading_ids], axis = 1).rename(columns = {0:'question_id'})
    
    return pd.DataFrame(questions_and_ids)

In [ ]:
#I created this to extract ALL answers from ALL survey responses from SurveyMonkey API, this has been replaced by another function in this list, get_details_
#choiceids_etc.
def get_bulk_answers(json):
    
    ''' 
    Takes JSON object returned from SurveyMonkey API and returns all answer ids and text answers for a survey
    along with the corresponding row_id
    
    Argument: JSON API called object
    '''
    
    bulk_response_answer_list = []
    for data in json['data']:
        for pages in data['pages']:
            for questions in pages['questions']:
                for answers in questions['answers']:
                    bulk_response_answer_list.append(answers.get('choice_id'))
                    
    bulk_response_answer_text_list = []
    for data in json['data']:
        for pages in data['pages']:
            for questions in pages['questions']:
                for answers in questions['answers']:
                    bulk_response_answer_text_list.append(answers.get('text'))
                    
    bulk_response_answer_list = pd.DataFrame(bulk_response_answer_list)
    bulk_response_answer_text_list = pd.DataFrame(bulk_response_answer_text_list)

    #now filling na values with each other
    bulk_responses_df = pd.concat([bulk_response_answer_list, bulk_response_answer_text_list], axis = 1)
    bulk_responses_df.columns = [['response_id', 'response_text']]

     #now fill in NA values with text from adjacent column to get ALL answers
    bulk_responses_df = pd.DataFrame(bulk_responses_df.bfill(axis=1).iloc[:, 0])
    
    #now get row_id
    bulk_response_row_id_list = []
    for data in json['data']:
        for pages in data['pages']:
            for questions in pages['questions']:
                for answers in questions['answers']:
                    bulk_response_row_id_list.append(answers.get('row_id'))
                    
    row_ids = pd.DataFrame(bulk_response_row_id_list)
    row_ids.columns = [['row_id']]

    #concatenate this column with all responses
    bulk_responses_with_row_ids = pd.concat([bulk_responses_df, row_ids], axis = 1)
    
    return bulk_responses_with_row_ids

In [1095]:
def get_answers_and_ids(json):
    
    '''
    This function takes a json object from the SurveyMonkey API, flattens the json, subsets the items in the json, and extracts
    the answers and their corresponding ids and then places all of this into a new dataframe
    
    Arguments: json object from SurveyMonkey API call
    '''
    
    #flatten DEEPLY nested JSON, same source as above
    from itertools import chain, starmap

    def flatten_json_iterative_solution(dictionary):
        """Flatten a nested json file"""
        def unpack(parent_key, parent_value):
            """Unpack one level of nesting in json file"""
        # Unpack one level only!!!
        
            if isinstance(parent_value, dict):
                for key, value in parent_value.items():
                    temp1 = parent_key + '_' + key
                    yield temp1, value
            elif isinstance(parent_value, list):
                i = 0 
                for value in parent_value:
                    temp2 = parent_key + '_'+str(i) 
                    i += 1
                    yield temp2, value
            else:
                yield parent_key, parent_value    

            
        # Keep iterating until the termination condition is satisfied
        while True:
            # Keep unpacking the json file until all values are atomic elements (not dictionary or list)
            dictionary = dict(chain.from_iterable(starmap(unpack, dictionary.items())))
            # Terminate condition: not any value in the json file is dictionary or list
            if not any(isinstance(value, dict) for value in dictionary.values()) and \
               not any(isinstance(value, list) for value in dictionary.values()):
                break

        return dictionary

    
    #use the function on the json
    flattened_details = pd.Series(flatten_json_iterative_solution(json)).to_frame().reset_index()
    
    flattened_details.rename(columns = {'index':'detail_buckets', 0:'details'}, inplace = True)
    
    
    #searching for all responses within the survey using regex
    choices = flattened_details[flattened_details['detail_buckets'].str.contains(r'questions_[0-9]{1,2}_answers_choices_[0-9]{1,2}_text|questions_[0-9]{1,2}_answers_other_text')].rename(columns = {'details':'possible_choices'})\
    .reset_index(drop = True)
    
    #searching for all response ids within the survey with regex
    choice_ids = flattened_details[flattened_details['detail_buckets'].str.\
    contains(r'pages_[0-9]{1,2}_questions_[0-9]{1,2}_answers_choices_[0-9]{1,2}_id|pages_[0-9]{1,2}_questions_[0-9]{1,2}_answers_other_id')].drop('detail_buckets', axis = 1).rename(columns = {'details':'response_id'}).reset_index(drop = True)

    final_df = pd.concat([choices, choice_ids], axis = 1)
    
    final_df.drop('detail_buckets', axis = 1, inplace = True)
    
    return final_df

In [ ]:
json = client.get_survey_details('insert survey id')

def get_row_text_and_row_ids(json):
    
    '''
    This function takes a json object returned from a call to the SurveyMonkey API and returns sub-questions and their ids
    
    Argument: a json object called from the SurveyMonkey API
    
    Requirements: the SurveyMonkey client - https://github.com/GearPlug/surveymonkey-python
    '''
    
    from itertools import chain, starmap

    def flatten_json_iterative_solution(dictionary):
        """Flatten a nested json file"""
        def unpack(parent_key, parent_value):
            """Unpack one level of nesting in json file"""
        # Unpack one level only!!!
        
            if isinstance(parent_value, dict):
                for key, value in parent_value.items():
                    temp1 = parent_key + '_' + key
                    yield temp1, value
            elif isinstance(parent_value, list):
                i = 0 
                for value in parent_value:
                    temp2 = parent_key + '_'+str(i) 
                    i += 1
                    yield temp2, value
            else:
                yield parent_key, parent_value    

            
        # Keep iterating until the termination condition is satisfied
        while True:
            # Keep unpacking the json file until all values are atomic elements (not dictionary or list)
            dictionary = dict(chain.from_iterable(starmap(unpack, dictionary.items())))
            # Terminate condition: not any value in the json file is dictionary or list
            if not any(isinstance(value, dict) for value in dictionary.values()) and \
               not any(isinstance(value, list) for value in dictionary.values()):
                break

        return dictionary
    
    
    flattened_details = pd.Series(flatten_json_iterative_solution(json)).to_frame().reset_index()
    
    flattened_details.rename(columns = {0:'row_id'}, inplace = True)
    
    row_text = flattened_details[flattened_details['index'].str.contains(r'pages_[0-9]{1,4}_questions_[0-9]{1,4}_answers_rows_[0-9]{1,4}_text|rows_[0-9]{1,4}_id')] 
    
    row_text_ids = row_text[row_text['row_id'].str.contains('1') == True]
    
    row_text = row_text[row_text['row_id'].str.contains('1') == False]

    row_id_df = pd.DataFrame(pd.concat([row_text, row_text_ids], axis = 1)['row_id'].iloc[:,1].dropna())
    
    row_text_df =  pd.DataFrame(pd.concat([row_text, row_text_ids], axis = 1)['row_id'].iloc[:,0].dropna())

    rows_df = pd.concat([row_text_df, row_id_df], axis = 1)

    rows_df.iloc[:,1] = rows_df.iloc[:,1].shift(-1)

    rows_df.dropna(inplace = True)

    rows_df.columns = ['row_text', 'row_id']
    
    return rows_df

In [1164]:
#json = client.get_survey_response_bulk('186927358')
#json_2 = client.get_survey_details('186927358')

def get_personid_choiceid_rowid_surveyid(json, json_2):
    
    '''
    
    So, theres a lot going on here... basically this function takes a json object from the SurveyMonkey API call and produces all answers, 
    their corresponding row_ids (which are question ids in the surveys), the respondent ids, the question ids, choice ids, and text answers
    
    Arguments: you must create 2 json objects from the client call first, 
        json = bulk responses from API
        json_2 = survey details
        
    '''
    import pandas as pd
    import numpy as np
    
    #use function to flatten DEEPLY nested JSON, same source as above
    from itertools import chain, starmap

    def flatten_json_iterative_solution(dictionary):
        """Flatten a nested json file"""
        def unpack(parent_key, parent_value):
            """Unpack one level of nesting in json file"""
        # Unpack one level only!!!
        
            if isinstance(parent_value, dict):
                for key, value in parent_value.items():
                    temp1 = parent_key + '_' + key
                    yield temp1, value
            elif isinstance(parent_value, list):
                i = 0 
                for value in parent_value:
                    temp2 = parent_key + '_'+str(i) 
                    i += 1
                    yield temp2, value
            else:
                yield parent_key, parent_value    

            
        # Keep iterating until the termination condition is satisfied
        while True:
            # Keep unpacking the json file until all values are atomic elements (not dictionary or list)
            dictionary = dict(chain.from_iterable(starmap(unpack, dictionary.items())))
            # Terminate condition: not any value in the json file is dictionary or list
            if not any(isinstance(value, dict) for value in dictionary.values()) and \
               not any(isinstance(value, list) for value in dictionary.values()):
                break

        return dictionary
    
    #getting the bulk responses and flattening the json file
    bulk_responses = pd.Series(flatten_json_iterative_solution(json)).to_frame().reset_index()
    
    #renaming the columns in the dataframe
    bulk_responses.rename(columns = {'index':'answer_type', 0:'answer'}, inplace = True)
    
    #searching for ids within the bulk responses, looks like I am only getting 50 unique back at a time...
    bulk_responses = bulk_responses[bulk_responses['answer_type'].str.contains(r'text|data_[0-9]{1,2}_id|data_[0-9]{1,2}_pages_[0-9]' \
    '{1,2}_questions_[0-9]{1,2}_id|data_[0-9]{1,2}_pages_[0-9]{1,2}_questions_[0-9]{1,2}_answers_[0-9]{1,2}_row_id|data_[0-9]{1,2}_pages_[0-9]' \
    '{1,2}_questions_[0-9]{1,2}_answers_[0-9]{1,2}_choice_id|other')].reset_index(drop = True)
    
    #grabbing the survey details from json_2
    survey_details = pd.Series(flatten_json_iterative_solution(json_2)).to_frame()
    
    #get survey id and create column denoting the survey id
    bulk_responses['survey_id'] = survey_details[survey_details.index.str.contains(r'^id') == True][0][0]

    #renameing the ids to normal names
    bulk_responses.replace(to_replace ='data_[0-9]{1,3}_id', value = 'respondent_id', regex = True, inplace = True)
    
    bulk_responses.replace(to_replace ='data_[0-9]{1,3}_pages_[0-9]{1,3}_questions_[0-9]{1,3}_answers_[0-9]{1,3}_text', 
                           value = 'text_answer', regex = True, inplace = True)
    
    bulk_responses.replace(to_replace ='data_[0-9]{1,3}_pages_[0-9]{1,3}_questions_[0-9]{1,3}_id', 
                           value = 'question_id', regex = True, inplace = True)
    
    bulk_responses.replace(to_replace ='data_[0-9]{1,3}_pages_[0-9]{1,3}_questions_[0-9]{1,3}_answers_[0-9]{1,3}_row_id', 
                           value = 'row_id', regex = True, inplace = True)
    
    bulk_responses.replace(to_replace ='data_[0-9]{1,3}_pages_[0-9]{1,3}_questions_[0-9]{1,3}_answers_[0-9]{1,3}_choice_id', 
                           value = 'choice_id', regex = True, inplace = True)
    
    #some surveys have "other" as an option, this covers those
    bulk_responses.replace(to_replace ='data_[0-9]{1,4}_pages_[0-9]{1,3}_questions_[0-9]{1,3}_answers_[0-9]{1,3}_other_id', 
                           value = 'other_id', regex = True, inplace = True)

    #fill in survey_id column completely
    bulk_responses.ffill(inplace = True)
    
    #create mask to use as a transfer from row ids to another column
    mask = (bulk_responses['answer_type'] == 'row_id')

    #use the mask
    bulk_responses['row_id'] = bulk_responses['answer_type'][mask]

    #set row id equal to the actual row id
    bulk_responses.row_id[bulk_responses.row_id == 'row_id'] = bulk_responses.answer

    #shift all row ids up one in order to drop the row ids from the details column
    bulk_responses['row_id'] = bulk_responses['row_id'].shift(-1)

    #drop all row ids rows from df so that row id is listed beside choice id
    bulk_responses = bulk_responses[~bulk_responses.answer_type.str.contains('row_id')]

    #rearrange columns
    bulk_responses = bulk_responses[['answer_type', 'answer', 'row_id', 'survey_id']]
    
    #create mask to use as a transfer to question ids column
    mask_2 = (bulk_responses['answer_type'] == 'question_id')

    #use the mask
    bulk_responses['question_id'] = bulk_responses['answer_type'][mask_2]

    #put actual question ids into the question ids column
    bulk_responses.question_id[bulk_responses.question_id == 'question_id'] = bulk_responses.answer

    #shift all of them down 1 to make sure answers line up next to actual questions
    bulk_responses['question_id'] = bulk_responses['question_id'].shift(1)

    #drop question id from answer type
    bulk_responses = bulk_responses[~bulk_responses.answer_type.str.contains('question_id')]

    #forward fill the question ids so that each row id has a corresponding question id
    bulk_responses['question_id'] = bulk_responses['question_id'].ffill()

    mask_3 = (bulk_responses['answer_type'] == 'respondent_id')

    bulk_responses['respondent_id'] = bulk_responses['answer_type'][mask_3]

    bulk_responses.respondent_id[bulk_responses.respondent_id == 'respondent_id'] = bulk_responses.answer

    bulk_responses['respondent_id'] = bulk_responses['respondent_id'].ffill()

    bulk_responses = bulk_responses[~bulk_responses['answer_type'].str.contains('respondent_id')]
    
    #change column order
    bulk_responses = bulk_responses[['respondent_id', 'survey_id', 'answer_type', 'answer', 'row_id', 'question_id']].reset_index(drop = True)
    
    return bulk_responses

# NLP Functions

In [ ]:
#cleans text for analysis
#https://towardsdatascience.com/detecting-bad-customer-reviews-with-nlp-d8b36134dc7e
# return the wordnet object value corresponding to the POS tag
from nltk.corpus import wordnet

def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer

def clean_text(text):
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    pos_tags = pos_tag(text)
    # lemmatize text
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return(text)

#example of use to clean a column of text in a df
reviews['tidy_reviews'] = reviews['text'].apply(lambda x: clean_text(x))

# Time Series Functions

In [ ]:
#this function is a complete augmented Dickey-Fuller test for stationarity, p<.05 means the data is stationary, taken from Jose Portilla's time series class
from statsmodels.tsa.stattools import adfuller

def adf_test(series,title=''):
    """
    Pass in a time series and an optional title, returns an ADF report
    """
    print(f'Augmented Dickey-Fuller Test: {title}')
    result = adfuller(series.dropna(),autolag='AIC') # .dropna() handles differenced data
    
    labels = ['ADF test statistic','p-value','# lags used','# observations']
    out = pd.Series(result[0:4],index=labels)

    for key,val in result[4].items():
        out[f'critical value ({key})']=val
        
    print(out.to_string())          # .to_string() removes the line "dtype: float64"
    
    if result[1] <= 0.05:
        print("Strong evidence against the null hypothesis")
        print("Reject the null hypothesis")
        print("Data has no unit root and is stationary")
    else:
        print("Weak evidence against the null hypothesis")
        print("Fail to reject the null hypothesis")
        print("Data has a unit root and is non-stationary")

# Stats Functions

In [ ]:
#get chronbachs alpha for subsets of measures
def cronbach_alpha(items):
    items = pd.DataFrame(items)
    items_count = items.shape[1]
    variance_sum = float(items.var(axis=0, ddof=1).sum())
    total_var = float(items.sum(axis=1).var(ddof=1))
    
    return (items_count / float(items_count - 1) *
            (1 - variance_sum / total_var))

In [ ]:
#I got really tired of running seperate lines of code for fifteen minutes just to run a few t tests, therefore I created this function to automate the process, 
# including checking for basic t test assumptions
def t_test(df, x, y, paired = False): 
    
    '''
    Takes two samples of data, runs Levene's Test to determine variance assumption veracity, then runs either 
    a dependent or independent samples T-Test with a printout report of effect sizes and summary statistics
    
    Argument format:
    df = data frame name 
    x = df['column1']
    y = df['column2']
    paired = False
    
    IMPORTANT NOTE: Categorical column data must be recoded to 1 and 2 prior to using the function for it to work properly
    '''
    
    import researchpy as rp
    from scipy.stats import levene
    
    #if the first sample is categorical (2 categories) then the categorical groups are tested against each other 
    if x.nunique() == 2:
        a = df[x == 1].iloc[:, 1]
        b = df[x == 2].iloc[:, 1]
        c, d = levene(a, b)
        print(levene(a, b))
        
        #if Levene's Test is significant, equal_var = False
        if d >= .05:
            print(f'T-Test Categorical Comparison')
            print(rp.ttest(a, b, equal_variances = False))
        else:
            print(f'T-Test Categorical Comparison')
            print(rp.ttest(a, b, equal_variances = True))
        return
    
    
    #if the t-test is paired samples (dependent), run the test and exit the function
    elif paired == True:
        e = rp.ttest(x, y, paired = True)
        return e
    else:
        
    #If the t-test is independent samples, run the Levene's Test and appropriate Welch's t-test with stated variance condition (True, False)
    
    #tuple unpacking to grab the p-value of Levene's Test
        f, g = levene(x, y)
        print(f"P-Value for Levene's Test: {g}")
        
        if g <= .05:
            h = rp.ttest(x, y, equal_variances = False)
            return f
        else:
            i = rp.ttest(x, y, equal_variances = True)
    return i

# Modeling

In [ ]:
#took this from Jeff Macaluso's github, but I use it all the time so I threw it in here, I need to create one for classification next
def get_score(model):
        '''
        Fits the model and returns a series containing the RMSE, MAE, and R^2
        '''
        from sklearn.metrics import mean_squared_error, mean_absolute_error
        import time

        startTime = time.time()  # Getting training time
        
        # Fits with training set
        model.fit(X_train, y_train)
        totalTime = time.time() - startTime
        predictions = model.predict(X_test)
        
        r2 = model.score(X_test, y_test)
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        mae = mean_absolute_error(y_test, predictions)
            
        score_results = pd.Series([r2, rmse, mae, totalTime], index=['R^2', 'RMSE', 'MAE', 'TrainingTime(sec)'])
        
        return score_results

# Frankenstein functions

In [3]:
def t_test(df, x, y, paired = False): 
    
    '''
    Takes two samples of data, runs Levene's Test to determine variance assumption veracity, then runs either 
    a dependent or independent samples T-Test with a printout report of effect sizes and summary statistics
    
    Argument format:
    df = data frame name 
    x = df['column1']
    y = df['column2']
    paired = False
    
    IMPORTANT NOTE: Categorical column data must be recoded to 1 and 2 prior to using the function for it to work properly
    '''
    
    import researchpy as rp
    from scipy.stats import levene
    
    #if the first sample is categorical (2 categories) then the categorical groups are tested against each other 
    if x.nunique() == 2 & paired == True:
        a = df[x == 1].iloc[:, 1]
        b = df[x == 2].iloc[:, 1]
        c, d = levene(a, b)
        print(levene(a, b))
        
        #if Levene's Test is significant, equal_var = False
        if d >= .05:
            print(f'T-Test Categorical Comparison')
            print(rp.ttest(a, b, equal_variances = False))
        else:
            print(f'T-Test Categorical Comparison')
            print(rp.ttest(a, b, equal_variances = True))
        return
    elif x.unique() == 2 & paired == False:
         a = df[x == 1].iloc[:, 1]
        b = df[x == 2].iloc[:, 1]
        c, d = levene(a, b)
        print(levene(a, b))
        
        #if Levene's Test is significant, equal_var = False
        if d >= .05:
            print(f'T-Test Categorical Comparison')
            print(rp.ttest(a, b, equal_variances = False))
        else:
            print(f'T-Test Categorical Comparison')
            print(rp.ttest(a, b, equal_variances = True))
        return
        
    
    #if the t-test is paired samples (dependent), run the test and exit the function
    elif paired == True:
        e = rp.ttest(x, y, paired = True)
        return e
    else:
        
    #If the t-test is independent samples, run the Levene's Test and appropriate Welch's t-test with stated variance condition (True, False)
    
    #tuple unpacking to grab the p-value of Levene's Test
        f, g = levene(x, y)
        print(f"P-Value for Levene's Test: {g}")
        
        if g <= .05:
            h = rp.ttest(x, y, equal_variances = False)
            return f
        else:
            i = rp.ttest(x, y, equal_variances = True)
    return i

In [83]:
def median(numbers):
	if len(numbers) < 1:
		return None
	elif len(numbers) % 2 == 1:
		return sorted(numbers)[int(len(numbers)/2)]
	else:
		return float(sorted(numbers)[int(len(numbers)/2) - 1] + sorted(numbers)[int(len(numbers)/2)])/2

## Visualization

In [ ]:
#create a wordcloud from a column of text
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
text = df.column.values
wordcloud = WordCloud(
    width = 2000,
    height = 1000,
    background_color = 'black',
    stopwords = STOPWORDS).generate(str(text))
fig = plt.figure(
    figsize = (40, 30),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

## Modeling

In [ ]:
def hierarchical_clustering(method, df, label_col, learning_rate, cluster_element):
    
    
    X = df.iloc[:, [1, 2]].values
    
    # Calculate the linkage: mergings
    mergings = linkage(X, method = method) #choose linkage method
    labels = list(df[label_col])
    

    # Plot the dendrogram, using varieties as labels
    plt.figure(figsize = (20, 8))
    dendrogram(mergings, labels = labels, leaf_rotation=90, leaf_font_size = 8)
    plt.title('Hierarchical Clustering Dendrogram', fontsize = 20);
    
    #instantiate T-SNE, learning_rate may need to be tweaked to get the visual right
    model = TSNE(learning_rate=learning_rate)

    #transform the data, T-SNE only has a fit_transform method! So, you cannot extend a T-SNE map to include new samples
    transformed = model.fit_transform(X)

    #define the axes
    xs = transformed[:,0]
    ys = transformed[:,1]

    #plot
    plt.figure(figsize = (15, 10))
    plt.scatter(xs, ys, c = cluster_element)  #cluster element is the variable you want to cluster on (i.e. type of plant in iris dataset)
    plt.title('T-SNE for Visualizing Clusters', fontsize = 16)

#     #if you wanted to annotate the points
#     for x, y, label in zip(xs, ys, label_name):
#         plt.annotate(label, (x, y), fontsize=8, alpha=0.75)
    
    plt.show()

    #example usage
#hierarchical_clustering('complete', df, 'country', 100, pairs['labels'])

# R Functions

## EDA

In [ ]:
# taken from stackoverflow, sorry I dont have the exact link

#view data side by side
sidebyside <- function(..., width=60){
  l <- list(...)
  p <- lapply(l, function(x){
        xx <- capture.output(print(x, width=width))
        xx <- gsub("\"", "", xx)
        format(xx, justify="left", width=width)
      }
  )
  p <- do.call(cbind, p)
  sapply(seq_len(nrow(p)), function(x)paste(p[x, ], collapse=""))
}

In [ ]:
#taken from stackoverflow, search for change column order

##arrange df vars by position
##'vars' must be a named vector, e.g. c("var.name"=1)
arrange.vars <- function(data, vars){
    ##stop if not a data.frame (but should work for matrices as well)
    stopifnot(is.data.frame(data))

    ##sort out inputs
    data.nms <- names(data)
    var.nr <- length(data.nms)
    var.nms <- names(vars)
    var.pos <- vars
    ##sanity checks
    stopifnot( !any(duplicated(var.nms)), 
               !any(duplicated(var.pos)) )
    stopifnot( is.character(var.nms), 
               is.numeric(var.pos) )
    stopifnot( all(var.nms %in% data.nms) )
    stopifnot( all(var.pos > 0), 
               all(var.pos <= var.nr) )

    ##prepare output
    out.vec <- character(var.nr)
    out.vec[var.pos] <- var.nms
    out.vec[-var.pos] <- data.nms[ !(data.nms %in% var.nms) ]
    stopifnot( length(out.vec)==var.nr )

    ##re-arrange vars by position
    data <- data[ , out.vec]
    return(data)
}